这个笔记用于记录langchain 自定义chain + callback 
> 感觉langchain 存在bug，document_retrive 没有生效，暂时先不处理了

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv('/Users/wushangcheng/project/tech-basis/AI/.env')

OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_API_VERSION = os.getenv('OPENAI_API_VERSION')
OPENAI_API_TYPE = os.getenv('OPENAI_API_TYPE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_CHAT_DEPLOY_NAME=os.getenv('OPENAI_CHAT_DEPLOY_NAME')

模型创建

In [ ]:

from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings

llm = AzureChatOpenAI(
  openai_api_base=OPENAI_API_BASE,
  openai_api_version=OPENAI_API_VERSION,
  deployment_name=OPENAI_CHAT_DEPLOY_NAME,
  openai_api_key=OPENAI_API_KEY,
  openai_api_type=OPENAI_API_TYPE,
  streaming=True,
  verbose=True,
)

embeddings = OpenAIEmbeddings(deployment="embedding", chunk_size=1)

加载文档 vectordb

In [ ]:

from langchain.vectorstores import Chroma
import chromadb
from chromadb.config import Settings

from langchain.document_loaders import DirectoryLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

def is_chroma_collection_exist(collection_name, persist_dir):
    print(collection_name, persist_dir)
    try:
        client = chromadb.Client(Settings(
            chroma_db_impl="duckdb+parquet",
            persist_directory=persist_dir
        ))
        client.get_collection(name=collection_name)
        return True
    except Exception as e:
        print(e)
        return False

def get_or_init_docs_collection(collection_name, docs_source_path, embedding, persist_dir):
    print('code in')
    try:
        if not is_chroma_collection_exist(collection_name=collection_name, persist_dir=persist_dir):
            loader = DirectoryLoader(docs_source_path, glob="**/*.md")
            docs = loader.load()
            print(f'You have {len(docs)} document(s) in your data')
            text_splitter = RecursiveCharacterTextSplitter(
                chunk_size=1000, chunk_overlap=0)
            split_docs = text_splitter.split_documents(docs)
            vectorstore = Chroma.from_documents(
                split_docs, embedding, persist_directory=persist_dir)
            vectorstore.persist()
            print(f'已为当前文档集生成矢量数据')
            vectorstore = None

        return Chroma(persist_directory=persist_dir, embedding_function=embedding, collection_name=collection_name)
    except Exception as e:
        print("####文档矢量数据库加载失败####")
        print(e)


document_v_db = get_or_init_docs_collection(
    collection_name='test_001_doc', 
    docs_source_path="/Users/wushangcheng/Desktop/Archive", 
    embedding=embeddings, 
    persist_dir="./store/doc"
)

加载聊天历史记录 vectordb

In [ ]:
# 判断是否存在该聊天分组的矢量数据库，不存在就新建一个
def get_or_create_chatgroup_vector_db(chat_id, embedding, persist_dir="store"):
    if not is_chroma_collection_exist(chat_id, persist_dir):
        print("正在使用历史聊天记录为当前聊天分组构建矢量数据库...")
        # TODO: retrive chat history from db
        chat_history = [
            "System: You are a assistant, you will answer Human questions, use the following pieces of context to answer the question . Try to give some answer Human may like."]
        vectordb = Chroma.from_texts(
            persist_directory=persist_dir, texts=chat_history, embedding=embedding, collection_name=chat_id)
        vectordb.persist()
        vectordb = None

    return Chroma(persist_directory=persist_dir, embedding_function=embedding, collection_name=chat_id)


chat_v_db = get_or_create_chatgroup_vector_db("test_001_chat", embeddings, './store/doc')

使用chat_c_db + doc_v_db 创建conversationRetrivalChain

In [ ]:

from langchain.memory import VectorStoreRetrieverMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks import get_openai_callback

chat_retriever = chat_v_db.as_retriever(search_kwargs=dict(k=3))
chat_memory = VectorStoreRetrieverMemory(retriever=chat_retriever, memory_key="chat_history")

chat_memory.save_context(
    {"Human": "什么是mysql数据库?"},
    {"Assistant": "MySQL是一种流行的开源关系型数据库管理系统（RDBMS），可以用于存储和管理各种类型的数据。它是一个客户端-服务器模型的数据库系统，支持多用户和多线程操作。MySQL具有较高的可扩展性、可靠性和灵活性，因此被广泛应用于Web应用程序、企业级应用程序等方面。MySQL支持多种编程语言，如C、C++、Java、Perl、Python等，并提供了许多API，例如ODBC、JDBC、.NET等，使得与其他应用程序的集成变得更加容易。MySQL还提供了许多高级功能，例如事务处理、视图、触发器、存储过程、用户定义的函数等，这些功能使得用户可以更加方便地管理和操作数据库。总之，MySQL是一个功能强大、灵活、易于使用的关系型数据库管理系统，适合各种规模和类型的应用程序。"}, 
    )
def get_chat_history(inputs) -> str:
    res = []
    for chat_log in inputs:
        res.append(f"{chat_log}")
    return "\n".join(res)

qa = ConversationalRetrievalChain.from_llm(
    llm, 
    document_v_db.as_retriever(), 
    memory=chat_memory, 
    get_chat_history=get_chat_history, 
    verbose=True
    )
result = qa({"question": "在docker 中如何创建它？"})
print('–––––––––––––––chat_question–––––––––––––––––––––')
print(result['question'])
print('–––––––––––––––chat_answer–––––––––––––––––––––')
print(result['answer'])

